# Cpastone


#### Import library and load the table from wiki

In [4]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df_table = pd.read_html(str(table))
df=pd.DataFrame(df_table[0])

In [5]:
df.shape

(287, 3)

In [6]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### remove the 'Not assigned'

In [7]:
df.Neighbourhood.replace('Not assigned',np.nan,inplace=True)
df.Borough.replace('Not assigned',np.nan,inplace=True)
df.dropna(axis=0,inplace=True)
df.head(5)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


#### reformat the data

In [8]:
df['new']=df.Postcode+df.Borough
dfn=df.groupby('new')['Neighbourhood'].apply(','.join).reset_index()
dfn.head()

,new,Neighbourhood
0,M1BScarborough,"Rouge,Malvern"
1,M1CScarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1EScarborough,"Guildwood,Morningside,West Hill"
3,M1GScarborough,Woburn
4,M1HScarborough,Cedarbrae


In [9]:
dfn['Postcode']=dfn.new.str[:3]
dfn['Borough']=dfn.new.str[3:]

In [14]:
dfn['Neighbourhood1']=dfn['Neighbourhood']
dfn.drop(['new','Neighbourhood'],axis=1,inplace=True)
dfn.head()

,Postcode,Borough,Neighbourhood1
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
dfn.rename(columns={'Neighbourhood1':'Neighbourhood'},inplace=True)


In [16]:
dfn.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
dfn.shape

(103, 3)

#### import the latitude and the longitude

In [21]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_5be308f4bf224a1b9a4991a87ddaee3d = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='uYTI6RGgoqL4F3E3D89N6f5tkv5HY-eNcyGZF45v1avR',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_5be308f4bf224a1b9a4991a87ddaee3d.get_object(Bucket='ibmcapston-donotdelete-pr-89ujnj0kn2h2pp',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
df_data_1.rename(columns={'Postal Code':'Postcode'},inplace=True)

####  merge two dataframe

In [25]:
df=pd.merge(left=dfn,right=df_data_1,on='Postcode')

In [27]:
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
